In [2]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import json
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

In [8]:
legal_boundary.head()

,SIG_CD,SIG_KOR_NM,geometry
0,11110,종로구,"MULTIPOLYGON (((127.00864 37.58047, 127.00871 ..."
1,11140,중구,"MULTIPOLYGON (((127.02314 37.57196, 127.02336 ..."
2,11170,용산구,"MULTIPOLYGON (((126.96918 37.55566, 126.96917 ..."
3,11200,성동구,"MULTIPOLYGON (((127.04341 37.57234, 127.04524 ..."
4,11215,광진구,"MULTIPOLYGON (((127.10166 37.5724, 127.10224 3..."


In [9]:

# 파일 경로 설정
path = '../processed_geojson_data/'
tbin = 'trash_bin_2024.geojson'
lb = 'legal_boundary.geojson'

# GeoJSON 파일 로드
trash_bins = gpd.read_file(path + tbin)  # 쓰레기통 위치
legal_boundary = gpd.read_file(path + lb)  # 법정 경계

# 좌표계 통일
trash_bins = trash_bins.to_crs(legal_boundary.crs)

# 공간 조인: 쓰레기통 좌표가 구 경계 안에 포함되는지 확인
trash_bins_with_districts = gpd.sjoin(trash_bins, legal_boundary, how="left", predicate="within")

# 결과 데이터에서 필요한 열 선택
result = trash_bins_with_districts[['geometry', 'SIG_KOR_NM']]

# 결과 저장 (옵션)
result.to_file(path + 'trash_bins_with_districts.geojson', driver='GeoJSON')

# 결과 확인
print(result.head())


                     geometry SIG_KOR_NM
0  POINT (126.97287 37.57616)        종로구
1  POINT (126.97287 37.57616)        종로구
2  POINT (126.97197 37.57856)        종로구
3  POINT (126.97197 37.57856)        종로구
4  POINT (126.97155 37.57997)        종로구


In [10]:
import geopandas as gpd

# 파일 경로 설정
path = '../processed_geojson_data/'
tbin = 'trash_bin_2024.geojson'
lb = 'legal_boundary.geojson'

# GeoJSON 파일 로드
trash_bins = gpd.read_file(path + tbin)  # 쓰레기통 위치
legal_boundary = gpd.read_file(path + lb)  # 법정 경계

# 좌표계 통일
trash_bins = trash_bins.to_crs(legal_boundary.crs)

# 공간 조인: 쓰레기통 좌표가 구 경계 안에 포함되는지 확인
trash_bins_with_districts = gpd.sjoin(trash_bins, legal_boundary, how="left", predicate="within")

# 결과 저장 (옵션)
# trash_bins_with_districts.to_file(path + 'trash_bins_with_districts.geojson', driver='GeoJSON')



              address   latitude   longitude                    geometry  \
0   서울특별시 종로구 사직로 125  37.576160  126.972866  POINT (126.97287 37.57616)   
1   서울특별시 종로구 사직로 125  37.576160  126.972866  POINT (126.97287 37.57616)   
2  서울특별시 종로구 자하문로 28   37.578560  126.971968  POINT (126.97197 37.57856)   
3   서울특별시 종로구 자하문로 28  37.578560  126.971968  POINT (126.97197 37.57856)   
4   서울특별시 종로구 자하문로 44  37.579972  126.971553  POINT (126.97155 37.57997)   

   index_right SIG_CD SIG_KOR_NM  
0          0.0  11110        종로구  
1          0.0  11110        종로구  
2          0.0  11110        종로구  
3          0.0  11110        종로구  
4          0.0  11110        종로구  


In [20]:
trash_bins_with_districts.dropna(axis = 0,inplace = True )
trash_bins_with_districts.isna().sum()


address       0
latitude      0
longitude     0
geometry      0
SIG_KOR_NM    0
dtype: int64

In [27]:
# 결과 확인
# trash_bins_with_districts = trash_bins_with_districts.drop(columns=['index_right','SIG_CD'])
display(trash_bins_with_districts.head())
trash_bins_with_districts.to_file(path + 'trash_bins_with_districts.geojson', driver='GeoJSON')

,address,latitude,longitude,geometry,SIG_KOR_NM
0,서울특별시 종로구 사직로 125,37.576160,126.972866,POINT (126.97287 37.57616),종로구
1,서울특별시 종로구 사직로 125,37.576160,126.972866,POINT (126.97287 37.57616),종로구
2,서울특별시 종로구 자하문로 28,37.578560,126.971968,POINT (126.97197 37.57856),종로구
3,서울특별시 종로구 자하문로 28,37.578560,126.971968,POINT (126.97197 37.57856),종로구
4,서울특별시 종로구 자하문로 44,37.579972,126.971553,POINT (126.97155 37.57997),종로구


In [21]:
# import folium
# from folium.plugins import MarkerCluster

# 이것 사용
# 중심 좌표 설정 (예: 서울시청 좌표)
center_lat, center_lon = 37.5665, 126.9780

# 폴리움 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# 1. 서울시 통합 경계 및 쓰레기통 레이어
seoul_layer = folium.FeatureGroup(name="서울시 통합 경계 및 쓰레기통", show=True)

# 서울시 전체 경계 추가
folium.GeoJson(
    legal_boundary,
    tooltip="서울시 경계"
).add_to(seoul_layer)

# 모든 쓰레기통 마커 추가
all_marker_cluster = MarkerCluster().add_to(seoul_layer)

for _, row in trash_bins_with_districts.iterrows():
    folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],
        tooltip=f"구: {row['SIG_KOR_NM']}"
    ).add_to(all_marker_cluster)

seoul_layer.add_to(m)

# 2. 구별 경계 및 쓰레기통 통합 레이어
for district_name in trash_bins_with_districts['SIG_KOR_NM'].unique():
    # FeatureGroup 생성
    district_layer = folium.FeatureGroup(name=f"{district_name}", show=False)
    
    # 구별 경계 추가
    district_boundary = legal_boundary[legal_boundary['SIG_KOR_NM'] == district_name]
    folium.GeoJson(
        district_boundary,
        tooltip=district_name  # 경계 툴팁
    ).add_to(district_layer)
    
    # 해당 구의 쓰레기통 데이터 필터링
    district_trash_bins = trash_bins_with_districts[trash_bins_with_districts['SIG_KOR_NM'] == district_name]
    marker_cluster = MarkerCluster().add_to(district_layer)
    
    for _, row in district_trash_bins.iterrows():
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            tooltip=f"구: {district_name}"
        ).add_to(marker_cluster)
    
    district_layer.add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 지도 저장 (옵션)
m.save('trash_bins_map_with_seoul_layer.html')

# 결과 지도 출력
# m
